In [1]:
import pandas as pd
import numpy as np
import re
import os

import plotly.express as px
import ipywidgets as widgets

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)

In [2]:
meta = pd.read_csv("./input_folder/merged/meta_final2.tsv", sep = "\t", index_col = 0)
rename_to = {v: k for k, v in meta["old_name"].to_dict().items()}

categories = ["Top", "Middle", "Bottom"]

# Convert the Series to Categorical with custom categories
meta["depthlayer"] = pd.Categorical(meta["depthlayer"], categories=categories, ordered=True)

meta.sort_values("depthlayer", inplace = True)

In [3]:
path = "input_folder/KS/16S_predicted.tsv.gz"
df = pd.read_csv(path, compression="gzip", header=0, sep="\t", index_col = 0)
df.sort_values("16S_rRNA_Count", ascending = False, inplace = True)

In [4]:
taxo = pd.read_csv("input_folder/KS/taxonomy.tsv", sep = "\t", index_col = 0)

## Taxonomy with NSTI value above 2

In [5]:
a = pd.DataFrame(
    taxo.loc[
            df.loc[df["metadata_NSTI"] > 2, :].index.tolist(), :
        ].value_counts("Taxon")
)

a.columns = ["Number of ASVs"]
a = a.loc[sorted(a.index.tolist()), :]
a.head()

,Number of ASVs
Taxon,
d__Archaea,3
d__Archaea; p__Aenigmarchaeota; c__Aenigmarchaeia; o__Aenigmarchaeales; f__Aenigmarchaeales; g__Aenigmarchaeales; s__uncultured_archaeon,4
d__Archaea; p__Aenigmarchaeota; c__Aenigmarchaeia; o__Aenigmarchaeales; f__Aenigmarchaeales; g__Candidatus_Aenigmarchaeum; s__uncultured_archaeon,2
d__Archaea; p__Crenarchaeota; c__Bathyarchaeia; o__Bathyarchaeia; f__Bathyarchaeia; g__Bathyarchaeia; s__uncultured_archaeon,1
d__Archaea; p__Nanoarchaeota; c__Nanoarchaeia; o__Woesearchaeales; f__GW2011_GWC1_47_15; g__GW2011_GWC1_47_15; s__uncultured_archaeon,1


In [6]:
a["Tax"] = [re.sub("; o__.*", "", i) for i in a.index.tolist()]
a.groupby("Tax").sum()

,Number of ASVs
Tax,
d__Archaea,3
d__Archaea; p__Aenigmarchaeota; c__Aenigmarchaeia,6
d__Archaea; p__Crenarchaeota; c__Bathyarchaeia,1
d__Archaea; p__Nanoarchaeota; c__Nanoarchaeia,14
d__Bacteria,1
d__Bacteria; p__Patescibacteria,4
d__Bacteria; p__Patescibacteria; c__ABY1,2
d__Bacteria; p__Patescibacteria; c__Microgenomatia,3
d__Bacteria; p__Patescibacteria; c__Parcubacteria,8


## Data transformation+ordination on predicted functions (Kegg Ontology)
### KS samples only

In [18]:
from IPython.display import display, clear_output

In [19]:
# Load your data
bray_pca_df = pd.read_csv("input_folder/KS/functional_metagenome/KO/bray_pca.tsv", sep="\t")
# meta = pd.read_csv("path_to_metadata.tsv", sep="\t")  # Example

# Dropdown for selecting metadata
metadata_dropdown_bray_pca = widgets.Dropdown(
    options=list(meta.columns), 
    description='Select Metadata:'
)

# Output widget to hold the plot
plot_output = widgets.Output()

# Function to update plot
def update_plot(change):
    with plot_output:
        clear_output(wait=True)
        selected_metadata = metadata_dropdown_bray_pca.value
        fig = px.scatter_3d(
            bray_pca_df, x='PC1', y='PC2', z='PC3',
            color=selected_metadata,
            text='uniqID',
            title='Bray Curtis + PCA Analysis on predicted KO'
        )
        fig.update_traces(textposition='top center')
        fig.update_layout(width=800, height=800)
        fig.show()

# Trigger update on dropdown change
metadata_dropdown_bray_pca.observe(update_plot, names='value')

# Initial plot
update_plot(None)

# Display in Voila
display(metadata_dropdown_bray_pca, plot_output)

Dropdown(description='Select Metadata:', options=('uniqID', 'filename_prefix', 'temperature', 'ph', 'orp', 'sa…

Output()

In [20]:
# Load data
bray_pca_df = pd.read_csv("input_folder/KS/functional_metagenome/KO/bray_pca.tsv", sep="\t")
# meta = pd.read_csv("path_to_metadata.tsv", sep="\t")  # Example if not yet defined

# Dropdown for selecting metadata
metadata_dropdown_bray_pca = widgets.Dropdown(
    options=list(meta.columns),
    description='Select Metadata:'
)

# Output widget to display the plot
plot_output_2d = widgets.Output()

# Update function for 2D PCA
def update_color_bray_pca(change):
    with plot_output_2d:
        clear_output(wait=True)
        selected_metadata = metadata_dropdown_bray_pca.value
        fig = px.scatter(
            bray_pca_df, x='PC1', y='PC2',
            color=selected_metadata,
            text='uniqID',
            title='Bray Curtis + PCA Analysis on predicted KO (2D)'
        )
        fig.update_traces(textposition='top center')
        fig.update_layout(width=800, height=800)
        fig.show()

# Trigger plot update on dropdown change
metadata_dropdown_bray_pca.observe(update_color_bray_pca, names='value')

# Initial plot render
update_color_bray_pca(None)

# Display the widgets and plot
display(metadata_dropdown_bray_pca, plot_output_2d)

Dropdown(description='Select Metadata:', options=('uniqID', 'filename_prefix', 'temperature', 'ph', 'orp', 'sa…

Output()

### Merged samples

In [21]:
# Load data from the updated path
bray_pca_df = pd.read_csv("input_folder/merged/functional_metagenome/KO/bray_pca.tsv", sep="\t")
# meta = pd.read_csv("path_to_metadata.tsv", sep="\t")  # Load this if not already done

# Dropdown for selecting metadata
metadata_dropdown_bray_pca = widgets.Dropdown(
    options=list(meta.columns),
    description='Select Metadata:'
)

# Output widget to display the plot
plot_output_3d = widgets.Output()

# Update function for 3D PCA
def update_color_bray_pca(change):
    with plot_output_3d:
        clear_output(wait=True)
        selected_metadata = metadata_dropdown_bray_pca.value
        fig = px.scatter_3d(
            bray_pca_df, x='PC1', y='PC2', z='PC3',
            color=selected_metadata,
            text='uniqID',
            title='Bray Curtis + PCA Analysis on predicted KO (3D)'
        )
        fig.update_traces(textposition='top center')
        fig.update_layout(width=1000, height=800)
        fig.show()

# Trigger plot update on dropdown change
metadata_dropdown_bray_pca.observe(update_color_bray_pca, names='value')

# Initial plot render
update_color_bray_pca(None)

# Display the dropdown and plot output
display(metadata_dropdown_bray_pca, plot_output_3d)

Dropdown(description='Select Metadata:', options=('uniqID', 'filename_prefix', 'temperature', 'ph', 'orp', 'sa…

Output()

In [22]:
# Dropdown for selecting metadata
metadata_dropdown_bray_pca = widgets.Dropdown(
    options=list(meta.columns),
    description='Select Metadata:'
)

# Output widget for displaying the plot
plot_output_2d = widgets.Output()

# Function to update the 2D plot
def update_color_bray_pca(change):
    with plot_output_2d:
        clear_output(wait=True)
        selected_metadata = metadata_dropdown_bray_pca.value
        fig = px.scatter(
            bray_pca_df, x='PC1', y='PC2',
            color=selected_metadata,
            text='uniqID',
            title='Bray Curtis + PCA Analysis on predicted KO (2D)'
        )
        fig.update_traces(textposition='top center')

        # Set fixed aspect ratio
        aspect_ratio = 0.6
        fig.update_layout(width=1200, height=int(1200 * aspect_ratio))

        fig.show()

# Trigger plot update when dropdown value changes
metadata_dropdown_bray_pca.observe(update_color_bray_pca, names='value')

# Initial plot render
update_color_bray_pca(None)

# Display the dropdown and plot
display(metadata_dropdown_bray_pca, plot_output_2d)

Dropdown(description='Select Metadata:', options=('uniqID', 'filename_prefix', 'temperature', 'ph', 'orp', 'sa…

Output()

## Metabolism heatmap from predicted functions (KO)

In [23]:
# Output widget for heatmap plot
plot_output = widgets.Output()

# Dropdown for KO subclass
ko_subclass_options = os.listdir("input_folder/ko_functions/")
ko_subclass_dd = widgets.Dropdown(
    options=ko_subclass_options,
    description="KO Subclass:"
)

# Initial module options based on first subclass
initial_modules = [re.sub(r"\.tsv$", "", f) for f in os.listdir(f"input_folder/ko_functions/{ko_subclass_options[0]}")]
modules_dd = widgets.Dropdown(
    options=initial_modules,
    description="Module:"
)

# Function to update heatmap
def update_metabolism_selection(*args):
    selected_subclass = ko_subclass_dd.value
    selected_module = modules_dd.value
    input_table = f"input_folder/ko_functions/{selected_subclass}/{selected_module}.tsv"

    with plot_output:
        clear_output(wait=True)
        try:
            sub_df = pd.read_csv(input_table, sep="\t", index_col=0)
            title = f"{selected_subclass} - {selected_module}"
            fig = px.imshow(sub_df, title=title)
            fig.update_layout(height=800)
            fig.show()
        except Exception as e:
            print(f"Error loading or plotting {input_table}: {e}")

# Function to update module options when KO subclass changes
def update_modules_options(change):
    selected_subclass = change['new']
    module_files = os.listdir(f"input_folder/ko_functions/{selected_subclass}")
    module_names = [re.sub(r"\.tsv$", "", f) for f in module_files]
    modules_dd.options = module_names
    modules_dd.value = module_names[0] if module_names else None
    update_metabolism_selection()

# Link callbacks
ko_subclass_dd.observe(update_modules_options, names='value')
modules_dd.observe(lambda change: update_metabolism_selection(), names='value')

# Initial plot
update_metabolism_selection()

# Display UI
display(ko_subclass_dd, modules_dd, plot_output)

Dropdown(description='KO Subclass:', options=('Methane_metabolism', 'Carbon_fixation'), value='Methane_metabol…

Dropdown(description='Module:', options=('M00345_Formaldehyde_assimilation_ribulose_monophosphate_pathway', 'M…

Output()

In [12]:
module_stats = pd.read_csv("input_folder/ko_modules_stats.tsv", sep="\t", index_col=0)
module_pred = pd.read_csv("input_folder/ko_modules_predictions.tsv", sep="\t", index_col=0)

# A metabolic pathway is associated with a number of KOs. Given all the predicted KOs, a score of each module is derived by summing from corresponding KOs.

## Are there any significant carbon fixation modules?

In [13]:
module_stats.loc[
    ["M00165", "M00168", "M00169", "M00172", "M00171", "M00170", "M00173", "M00376", "M00375", "M00374", "M00377", "M00579", "M00620"],
    :
]

,P-value,Corrected P-value,Effect Size,Description
M00165,0.000129,0.000538,0.668205,Reductive_pentose_phosphate_cycle_Calvin_cycle
M00168,0.000017,0.000118,0.871353,CAM_Crassulacean_acid_metabolism_dark
M00169,0.000023,0.000145,0.839846,CAM_Crassulacean_acid_metabolism_light
M00172,0.000004,0.000053,1.025887,C4-dicarboxylic_acid_cycle_NADP_-_malic_enzyme_type
M00171,0.000045,0.000217,0.772380,C4-dicarboxylic_acid_cycle_NAD_-_malic_enzyme_type
M00170,0.002677,0.006462,0.402778,C4-dicarboxylic_acid_cycle_phosphoenolpyruvate_carboxykinase_type
M00173,0.405146,0.476380,0.052985,Reductive_citrate_cycle_Arnon-Buchanan_cycle
M00376,0.000371,0.001240,0.570540,3-Hydroxypropionate_bi-cycle
M00375,0.019088,0.033482,0.253841,Hydroxypropionate-hydroxybutylate_cycle
M00374,0.117821,0.167348,0.129986,Dicarboxylate-hydroxybutyrate_cycle


## Are there any significant energy metabolism modules?

In [14]:
module_stats.loc[
    ["M00567", "M00357", "M00356", "M00563", "M00358", "M00608", "M00174", "M00346", "M00345", "M00344", "M00378", "M00935", "M00422"],
    :
]

,P-value,Corrected P-value,Effect Size,Description
M00567,0.019712,0.034389,0.251537,Methanogenesis_CO2_to_methane
M00357,0.009344,0.017562,0.306081,Methanogenesis_acetate_to_methane
M00356,0.007698,0.014976,0.320622,Methanogenesis_methanol_to_methane
M00563,0.009229,0.017562,0.307003,Methanogenesis_methylaminedimethylaminetrimethylamine_to_methane
M00358,0.001377,0.003843,0.457124,Coenzyme_M_biosynthesis
M00608,0.442983,0.510488,0.047626,2-Oxocarboxylic_acid_chain_extension_2-oxoglutarate_to_2-oxoadipate_to_2-oxopimelate_to_2-oxosuberate
M00174,0.371815,0.445345,0.058163,Methane_oxidation_methanotroph_methane_to_formaldehyde
M00346,0.000037,0.000191,0.791534,Formaldehyde_assimilation_serine_pathway
M00345,0.009412,0.017565,0.305539,Formaldehyde_assimilation_ribulose_monophosphate_pathway
M00344,0.000002,0.000036,1.117034,Formaldehyde_assimilation_xylulose_monophosphate_pathway


In [15]:
grouping_meta = "depthlayer"

groups_dict = {
    layer: meta[meta[grouping_meta] == layer].index.tolist() for layer in meta[grouping_meta].unique()
}
groups = groups_dict.values()

In [16]:
modules_dd = widgets.Dropdown(options=module_stats.index + ": " + module_stats["Description"])

def update_module_boxplot(selected_module):
    module_id = selected_module.split(":")[0]
    module_name = module_pred.loc[module_id, "Description"]
    boxval = [module_pred.loc[module_id, group].to_list() for group in groups]
    boxval = [i for b in boxval for i in b]

    index = [i for group in groups for i in group]

    boxval_df = pd.DataFrame(boxval, index = index, columns = ["Value"])
    boxval_df["Group"] = boxval_df.index.map({i: k for k, v in groups_dict.items() for i in v})
    boxval_df["Samples"] = boxval_df.index
    
    corrected_p_value, effect_size = module_stats.loc[module_id, ["Corrected P-value", "Effect Size"]]
    # corrected_p_value = round(corrected_p_value, 3)
    corrected_p_value = "{:.3E}".format(corrected_p_value)
    effect_size = round(effect_size, 3)
    
    
    fig = px.box(boxval_df, x="Group", y="Value", points="all", hover_data=["Samples"])
    fig.update_layout(title={'text': f"""<b>{module_id}</b>\t{module_name}</br></br><b>Corrected P-value</b>: {corrected_p_value}\t\t\t<b>Effect size</b>: {effect_size}"""})
    fig.show()
    
widgets.interactive(update_module_boxplot, selected_module=modules_dd)

interactive(children=(Dropdown(description='selected_module', options=('M00001: Glycolysis_Embden-Meyerhof_pat…